In [3]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO write the description
'''

datadir = "/datasets/eurora_data/db_local/"
#datadir = "/datasets/eurora_data/db/"
#datadir = "C:/Users/folid/git-repo/hpc_job_power_prediction/datasets/eurora_data/db_local/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"
infile_nodes = datadir + "nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
interval_comment_whole = "WholeData"

## 2 settings Andrea
#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')

infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"
#infile_jobs = datadir + "CPUs/" + interval_comment + "/Alina_jobs_real_pow_t2_54_234192"

#infile_jobs = datadir + "CPUs/jobs_cleaned_whole"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
#nodes=['01']
gpu_nodes = ['33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
mic_nodes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']

Along the nodes we have three differentIntel Xeon E5 processors stepping: nodes 1-16 & 25-32 havea maximum frequency of 2.1GHz (CPUs-2100), nodes 17-24 have a maximum frequency of 2.2GHz (CPUs-2200), andnodes 33-64 have a maximum frequency of 3.1GHz (CPUs-3100).

In [87]:
nodes_data = pd.read_csv(infile_nodes, index_col=0)
# nodes_data

### Read job2nodes data

In [88]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


#### solve the problem of jobs runned on node 129

In [89]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 469020 records


#### solve the problem of duplicates job_id_string, node_id
this is probably caused by the concurrency writing of the database 

In [90]:
#jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
#print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### Read jobs data

drop the jobs out of interval

In [91]:
jobs_whole = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("jobs_whole_data contains {} records".format(jobs_whole.shape[0]))

jobs_whole_data contains 336787 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [92]:
interval_jobs = jobs_whole[pd.to_datetime(jobs_whole['end_time']) <= end_time]
interval_jobs = interval_jobs[pd.to_datetime(interval_jobs['run_start_time']) > start_time]
print("Alina set contains {} records".format(interval_jobs.shape[0]))

Alina set contains 336787 records


#### merge

In [93]:
merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, interval_jobs, on='job_id_string')
print(merged_jobs_to_nodes.shape[0])

381008


In [94]:
### remove to continue from a specific job
##long_jobs = long_jobs.drop(['real_pow', 'runned_alone', 'real_pow_quality'], axis=1) #old?

interval_jobs['real_pow_acc'] = 0.0
#interval_jobs['ran_alone'] = True
#interval_jobs['real_pow_quality'] = 0.0

In [95]:
interval_jobs.head()

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,deleted,dependency,exit_status,pbs_exit_code,gpu_req,mic_req,real_pow_acc
job_id,,,,,,,,,,,,,,,,,,
1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,04:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,0.0
2,498459.node129,CPMD,parallel,2014-03-31 00:36:51,2014-03-31 00:36:51,2014-03-31 00:40:13,framondo@node129.eurora.cineca.it,1,16,14,01:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,0.0
3,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0@node129.eurora.cineca.it,6,96,84,04:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,0.0
4,498461.node129,rth2e6t0DC,parallel,2014-03-31 01:00:01,2014-03-31 01:00:02,2014-03-31 05:01:18,alupi000@node129.eurora.cineca.it,1,16,14,04:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,0.0
5,498462.node129,rth2e6t0,parallel,2014-03-31 01:30:02,2014-03-31 01:30:02,2014-03-31 02:24:29,alupi000@node129.eurora.cineca.it,1,16,14,04:00,NaN,NaN,COMPLETED,NaN,NaN,NaN,0.0


In [96]:
#debug
merged_jobs_to_nodes.head(1)

,node_id,job_id_string,ncpus,ngpus,nmics,mem_requested,job_name,queue,start_time,run_start_time,...,node_req,cpu_req,mem_req,time_req,deleted,dependency,exit_status,pbs_exit_code,gpu_req,mic_req
0,17,498458.node129,16,0,0,14680064,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,...,1,16,14,04:00,NaN,NaN,COMPLETED,NaN,NaN,NaN


In [97]:
import time

start_time = time.time()
i = 0
#i = 234193 # remove to continue from a specific job
outfile = datadir + 'CPUs/' + interval_comment + "/" + interval_comment + "_jobs_real_pow"

for node in nodes:
    # recupero le allocazioni nei vari nodi
    merged_jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
    print("---------- {} jobs in node {}".format(merged_jobs_to_node.shape[0], node))
    infile_node = datadir + "CPUs/" + interval_comment + "/node" + node + suffix + interval_comment + "_active_cores_and_jobs.csv"
    node_measurements = pd.read_csv(infile_node)
    node_measurements['timestamp'] = pd.to_datetime(node_measurements['timestamp'])
    node_start_time = pd.to_datetime(node_measurements['timestamp'].iloc[0])
    num_minutes = node_measurements.shape[0]
    print("-------------------node data starts at {}, num minute {}".format(node_start_time, num_minutes))
    
    # per ogni nodo recupero ogni allocazione
    for index_job_to_node, row_job_to_node in merged_jobs_to_node.iterrows():
        
                
        #print()
        if(i % 1000 == 0):
            print("{}/{}".format(i, merged_jobs_to_nodes.shape[0]))
        i = i + 1
            
        ### verifico che il nodo usi uno tra gpu e mic, altrimenti passo al prossimo
        if node in gpu_nodes:
            col_name = "gpus_pow"
            if (row_job_to_node['ngpus'] == 0):
                continue
            job_n_acc = row_job_to_node['ngpus']
                
        if node in mic_nodes:
            col_name = "mics_pow"
            if (row_job_to_node['nmics'] == 0):
                continue
            job_n_acc = row_job_to_node['nmics']

        
        partial_pow_consumption = 0
        job_id_string = row_job_to_node['job_id_string']
        
        ## recupero indice del job in interval_jobs
        job = interval_jobs[interval_jobs['job_id_string'] == job_id_string]
        job_index = interval_jobs[interval_jobs['job_id_string'] == job_id_string].index
        if(job_index != None):
            ## recupero eventuali valori scritti in precedenti iterazioni
            job_real_pow_acc = job['real_pow_acc'].iloc[0] 
            
            job_start_time = pd.to_datetime(row_job_to_node['run_start_time'])
            job_end_time = pd.to_datetime(row_job_to_node['end_time'])
            
            if(job_end_time - job_start_time >= np.timedelta64(5, 's')):
                before_minutes = int((job_start_time - node_start_time) / np.timedelta64(5, 's'))
                running_minutes = int((job_end_time - job_start_time) / np.timedelta64(5, 's'))
                after_minutes = num_minutes - running_minutes - before_minutes
                
                before_serie = pd.Series(False, index=np.arange(before_minutes))
                running_serie = pd.Series(True, index=np.arange(running_minutes))
                after_serie = pd.Series(False, index=np.arange(after_minutes))
                concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
                
                interval_measurements = node_measurements[concat_series]
                
                interval_measurements = interval_measurements[interval_measurements[col_name] != 0]
                interval_measurements = interval_measurements[np.isfinite(interval_measurements[col_name])]
                
                ### group the intervals wehere the partial_pow_cons can be obtaied with the same instance of the formula
                ### take the mean for the pow columns
                # interval_grouped = interval_measurements.reset_index().groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"]).mean()
                interval_grouped = interval_measurements.groupby(["active_jobs", "active_gpus", "active_mics"])
                counts = interval_grouped.size().to_frame(name='counts')
                interval_grouped = (counts
                 .join(interval_grouped.agg({col_name:'mean'}).rename(columns={col_name: 'acc_pow_mean'}))
                 .reset_index()
                )
                interval_grouped.sort_values('active_jobs')
                #print(interval_grouped)

                if(interval_grouped.shape[0]>1):
                    job_ran_alone = False


                if(interval_grouped.shape[0] != 0):
                    interval_grouped[col_name] = interval_grouped['acc_pow_mean'] / interval_grouped['active_jobs'] 

                    # not_alone_counts = interval_grouped['counts'].loc[interval_grouped['active_jobs'] != 1].sum()
                    # interval_grouped.loc[interval_grouped['active_jobs'] == 1, ['counts']] += not_alone_counts
                    #print(interval_grouped)
                    partial_pow_consumption = np.average(interval_grouped[col_name], weights=interval_grouped['counts'])
                    # partial_pow_consumption = partial_pow_consumption / interval_grouped.shape[0]
                    # print(interval_grouped)

                #print("partial measurement: {}".format(partial_pow_consumption))
                job_real_pow_acc += partial_pow_consumption
                partial_pow_consumption = 0

        
            else: # jobs shorter than 5 sec
                   # keep the only timepoint
                job_start_time = row_job_to_node['run_start_time']
                #job_end_time = row_job_to_node['end_time']
                if(int(job_start_time[18:]) >= 5):
                    job_start_time = pd.to_datetime(job_start_time[:-1] + "5")
                else:
                    job_start_time = pd.to_datetime(job_start_time[:-1] + "0")
                #if(int(job_end_time[18:]) >= 5):
                #    job_end_time = pd.to_datetime(job_end_time[:-1] + "0") + np.timedelta64(10, 's')
                #else:
                #    job_end_time = pd.to_datetime(job_end_time[:-1] + "5")
                
                measure = node_measurements[node_measurements['timestamp'] == job_start_time]
                #job_tot_timepoints += 1
                
                measure = measure.dropna()
                if(measure.shape[0] != 0):
                    
                    #job_good_nodes += 1
                    #if(measure['active_cores'].iloc[0] != 0):
                    #    job_ran_alone = False

                    active_jobs = measure['active_jobs'].iloc[0] + job_n_acc
                    #if(active_cores > 16):
                    #    active_cores = 16
                    job_real_pow_acc += measure[col_name].iloc[0] / active_jobs
              
            
            if (job_real_pow_acc < 0):
                job_real_pow_acc = 0.0
                
            #print("job_real_pow_acc: {}. {}".format(job_real_pow_acc, job_n_acc))
            
            interval_jobs.at[job_index, 'real_pow_acc'] = job_real_pow_acc
            #interval_jobs.at[job_index, 'n_2_1'] = job_n_2_1
            #interval_jobs.at[job_index, 'n_2_2'] = job_n_2_2
            #interval_jobs.at[job_index, 'n_3_1'] = job_n_3_1
            #interval_jobs.at[job_index, 'job_tot_timepoints'] = job_tot_timepoints
            #interval_jobs.at[job_index, 'job_timepoints'] = job_timepoints
            #interval_jobs.at[job_index, 'good_nodes'] = job_good_nodes
            #interval_jobs.at[job_index, 'ran_alone'] = job_ran_alone
    
        
        
    #if(i % 10000 == 0):
    interval_jobs.to_csv(outfile + "_" + node + "_" + str(i) + ".csv")
    print("--- %s seconds ---" % (time.time() - start_time))
        
interval_jobs.to_csv(outfile + "_" + node + "_" + str(i) + ".csv")

---------- 1746 jobs in node 01
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
0/381008
1000/381008
--- 75.09539103507996 seconds ---
---------- 240 jobs in node 02
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
--- 95.32685542106628 seconds ---
---------- 1625 jobs in node 03
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
2000/381008
3000/381008
--- 123.07743191719055 seconds ---
---------- 1775 jobs in node 04
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
4000/381008
5000/381008
--- 147.1897177696228 seconds ---
---------- 552 jobs in node 05
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
--- 164.38458514213562 seconds ---
---------- 1570 jobs in node 06
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
6000/381008
7000/381008
--- 185.4821937084198 seconds ---
---------- 1714 jobs in node 07
-----

141000/381008
142000/381008
143000/381008
144000/381008
145000/381008
146000/381008
147000/381008
148000/381008
149000/381008
150000/381008
151000/381008
152000/381008
153000/381008
154000/381008
155000/381008
156000/381008
157000/381008
158000/381008
159000/381008
160000/381008
161000/381008
162000/381008
163000/381008
164000/381008
165000/381008
166000/381008
167000/381008
168000/381008
169000/381008
170000/381008
171000/381008
172000/381008
173000/381008
174000/381008
175000/381008
176000/381008
177000/381008
178000/381008
179000/381008
180000/381008
181000/381008
182000/381008
183000/381008
184000/381008
185000/381008
186000/381008
--- 10870.47729420662 seconds ---
---------- 7585 jobs in node 46
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
187000/381008
188000/381008
189000/381008
190000/381008
191000/381008
192000/381008
193000/381008
194000/381008
--- 11359.90918302536 seconds ---
---------- 17507 jobs in node 47
-------------------node data sta

In [1]:
interval_jobs[interval_jobs['real_pow_acc'] > 0.0]

NameError: name 'interval_jobs' is not defined

In [101]:
infile_jobs = datadir + "CPUs/" + interval_comment + "/Alina_jobs_real_pow_64_381008_cpu"
jobs_whole = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("jobs_whole_data contains {} records".format(jobs_whole.shape[0]))

jobs_whole_data contains 336787 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [102]:
interval_jobs.shape

(336787, 18)

In [103]:
jobs_whole['real_pow_acc'] = interval_jobs['real_pow_acc']

In [104]:
outfile = datadir + "CPUs/" + interval_comment + "/Alina_jobs_real_pow" 
jobs_whole.to_csv(outfile + ".csv")

### nodes default calculation

In [53]:
##execute the first cell first
infile_jobs = datadir + "CPUs/" + interval_comment + "/Alina_jobs_real_pow_cleaned" 
jobs_whole = pd.read_csv(infile_jobs + ".csv", index_col=0)

In [55]:
jobs_whole.shape

(311113, 27)

In [56]:
job_gpu = jobs_whole[jobs_whole['gpu_req'] > 0]
real_pow_acc = job_gpu['real_pow_acc'] / job_gpu['gpu_req']
mean_pow_acc = real_pow_acc[real_pow_acc > 0].mean()

In [57]:
jobs_whole.loc[(jobs_whole['gpu_req'] > 0) & (jobs_whole['real_pow_acc'] == 0.0)]['real_pow_acc'].sum()

0.0

In [60]:
jobs_whole.loc[(jobs_whole['gpu_req'] > 0) & (jobs_whole['real_pow_acc'] == 0.0), 'real_pow_acc'] = jobs_whole.loc[(jobs_whole['gpu_req'] > 0) & (jobs_whole['real_pow_acc'] == 0.0)]['gpu_req']*mean_pow_acc

In [61]:
jobs_whole.loc[(jobs_whole['gpu_req'] > 0) & (jobs_whole['real_pow_acc'] == 0.0)]

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,real_pow,ran_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes,real_pow_acc
job_id,,,,,,,,,,,,,,,,,,,,,


In [62]:
# mic

In [63]:
job_mic = jobs_whole[jobs_whole['mic_req'] > 0]
real_pow_acc = job_mic['real_pow_acc'] / job_mic['mic_req']
mean_pow_acc = real_pow_acc[real_pow_acc > 0].mean()
print(mean_pow_acc)

199.3882113197929


In [64]:
jobs_whole.loc[(jobs_whole['mic_req'] > 0) & (jobs_whole['real_pow_acc'] == 0.0), 'real_pow_acc'] = jobs_whole.loc[(jobs_whole['mic_req'] > 0) & (jobs_whole['real_pow_acc'] == 0.0)]['mic_req']*mean_pow_acc

In [67]:
jobs_whole.loc[(jobs_whole['mic_req'] > 0) & (jobs_whole['real_pow_acc'] == 0.0)]

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,real_pow,ran_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes,real_pow_acc
job_id,,,,,,,,,,,,,,,,,,,,,


In [66]:
outfile = datadir + "CPUs/" + interval_comment + "/Alina_jobs_real_pow_cleaned" 
jobs_whole.to_csv(outfile + ".csv")

In [70]:
jobs_whole.loc[(jobs_whole['gpu_req'] > 0) & (jobs_whole['real_pow_acc'] < 2)]

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,real_pow,ran_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes,real_pow_acc
job_id,,,,,,,,,,,,,,,,,,,,,
205795,1006765.node129,442_1.pbs,parallel,2014-08-13 08:40:57,2014-08-13 10:19:44,2014-08-13 10:19:45,cdongmof@node129.eurora.cineca.it,1,4,1048576,...,99.704985,False,0.0,0,0,1,1,0,1,1.871786
205796,1006766.node129,442_2.pbs,parallel,2014-08-13 08:40:57,2014-08-13 10:19:47,2014-08-13 10:19:49,cdongmof@node129.eurora.cineca.it,1,4,1048576,...,97.160311,False,0.0,0,0,1,1,0,1,1.894357
205797,1006767.node129,442_3.pbs,parallel,2014-08-13 08:40:57,2014-08-13 10:19:49,2014-08-13 10:19:51,cdongmof@node129.eurora.cineca.it,1,4,1048576,...,97.160311,False,0.0,0,0,1,1,0,1,1.894357
205798,1006768.node129,442_4.pbs,parallel,2014-08-13 08:40:57,2014-08-13 10:19:51,2014-08-13 10:19:53,cdongmof@node129.eurora.cineca.it,1,4,1048576,...,97.126637,False,0.0,0,0,1,1,0,1,1.867857
205799,1006769.node129,442_5.pbs,parallel,2014-08-13 08:40:58,2014-08-13 10:19:54,2014-08-13 10:19:55,cdongmof@node129.eurora.cineca.it,1,4,1048576,...,97.126637,False,0.0,0,0,1,1,0,1,1.867857
205802,1006772.node129,406_1.pbs,parallel,2014-08-13 08:44:21,2014-08-13 10:20:06,2014-08-13 10:20:08,cdongmof@node129.eurora.cineca.it,1,4,1048576,...,98.099605,False,0.0,0,0,1,1,0,1,1.879286
205803,1006773.node129,406_2.pbs,parallel,2014-08-13 08:44:22,2014-08-13 10:20:09,2014-08-13 10:20:10,cdongmof@node129.eurora.cineca.it,1,4,1048576,...,98.099605,False,0.0,0,0,1,1,0,1,1.879286
205804,1006774.node129,406_3.pbs,parallel,2014-08-13 08:44:22,2014-08-13 10:20:11,2014-08-13 10:20:13,cdongmof@node129.eurora.cineca.it,1,4,1048576,...,97.625215,False,0.0,0,0,1,1,0,1,1.900500
205805,1006775.node129,406_4.pbs,parallel,2014-08-13 08:44:22,2014-08-13 10:20:13,2014-08-13 10:20:15,cdongmof@node129.eurora.cineca.it,1,4,1048576,...,97.625215,False,0.0,0,0,1,1,0,1,1.900500
